In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
import requests
import os
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path
from datetime import datetime
import sqlite3

In [2]:
# table and querry string dictionary
# querry_dic = {MID:innistrad-midnight-hunt,10E:10th-edition, BRO:the-brothers-war, DGM:dragons-maze,
# DMU:dominaria-united, ELD:throne-of-eldraine, GTC:gatecrash, MAT:march-of-the-machine-the-aftermath, 
#M   MOM:march-of-the-machine, NEO:kamigawa-neon-dynasty, ONE:phyrexia-all-will-be-one, RTR:return-to-ravnica,
#  SNC:streets-of-new-capenna, VOW:innistrad-crimson-vow, WOE: wilds-of-eldraine  }

In [3]:
from splinter import Browser
from selenium.webdriver.chrome.service import Service

my_service = Service(executable_path=r'C:\Users\pc\OneDrive\Desktop\TCG-scraper\chromedriver.exe')
browser = Browser('chrome', service=my_service)

In [4]:
dbtable ="WOE"
url_string= "wilds-of-eldraine"

In [5]:
# visit browser and make soup
url =f"https://shop.tcgplayer.com/price-guide/magic/{url_string}"
browser.visit(url)
browser.driver.maximize_window()

In [6]:
time.sleep(5)

In [7]:
req = browser.html

soup = BeautifulSoup(req, "html")

In [8]:
names= []
title_tags = soup.find_all('a', class_ = "pdp-url")
#title_tags
for i in title_tags:
   names.append(i.text)

In [9]:
prices =[]
price = soup.find_all('td', class_ = "tcg-table-body__cell tcg-table-body__cell--align-right")
#price
count = -1
for i in price:
    count = count +1
    if  count%2 > 0 :
        prices.append(i.text)

In [10]:
# turn lists into new data frame
price_list= zip(names, prices)
mkt_df = pd.DataFrame(price_list,columns = ['Name', 'Market Price']) 
mkt_df
#mkt_df.columns = ['Name', 'Market Price']
mkt_df.head(3)

,Name,Market Price
0,A Tale for the Ages,$0.16
1,A Tale for the Ages (Extended Art),$0.25
2,Agatha of the Vile Cauldron,$1.01


In [11]:
mkt_df['date']= datetime.today().strftime('%Y-%m-%d')
mkt_df.head(3)

,Name,Market Price,date
0,A Tale for the Ages,$0.16,2023-11-28
1,A Tale for the Ages (Extended Art),$0.25,2023-11-28
2,Agatha of the Vile Cauldron,$1.01,2023-11-28


In [12]:
file_path= ("../Resources/inventory.db")

In [13]:
price =  Path("../Resources/inventory.db")
con = sqlite3.connect(price)
cur = con.cursor()

In [14]:
q = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in q.fetchall():
    print(name[0])

10E
BRO
DGM
DMU
ELD
GTC
MAT
MID
MOM
NEO
ONE
RTR
SNC
VOW
WOE


In [15]:
db_df = pd.read_sql_query(f"SELECT * FROM '{dbtable}'", con)

db_df["MktPrice"] = 0
db_df.head(1)

,setCode,name,rarity,manaCost,colorIdentity,types,subtypes,power,toughness,quantity,nm,lp,p,foil,text,MktPrice
0,WOE,Archon of the Wild Rose,rare,{2}{W}{W},W,Creature,Archon,4,4,0,0,0,0,0,Flying\nOther creatures you control that are e...,0


In [16]:
#cur.execute(f"ALTER TABLE '{dbtable}' ADD COLUMN  MktPrice VARCHAR(10)")

In [17]:
# Match inventory df  to scrapped data and append current Market price to inventory df
count = 0
while count < len(db_df): # next itteration if loop fails 
    try:
        for i in range(count , len(db_df)):
            search = db_df.loc[i,['name']]   # 
            search = search['name']
            search_df = mkt_df.loc[mkt_df['Name'] == search]
                           # break if no match
            search_df.reset_index(drop=True, inplace=True)
            search_result = search_df["Market Price"][0]
            db_df.loc[i,['MktPrice']] = search_result
            count = count + 1
    except: # if no match 
            db_df.loc[i,['MktPrice']] = "na"
            count = count + 1

In [18]:
                        # change db table mtkPrice to set_df["Mktprice"]

for i in range(len(mkt_df)):
    search = mkt_df.loc[i,['Name']]   # 
    search = search['Name']
    x = mkt_df.loc[i,['Market Price']]  
    x = x['Market Price']
    insert_sql = f'''UPDATE  '{dbtable}'  set MktPrice ="{x}" where "name" = "{search}"'''
    cur.execute(insert_sql)
    con.commit()

In [19]:
set_df = pd.read_sql_query(f"SELECT * FROM '{dbtable}'", con)
set_df.head(280)

,setCode,name,rarity,manaCost,colorIdentity,types,subtypes,power,toughness,quantity,nm,lp,p,foil,text,MktPrice
0,WOE,Archon of the Wild Rose,rare,{2}{W}{W},W,Creature,Archon,4,4,0,0,0,0,0,Flying\nOther creatures you control that are e...,$0.18
1,WOE,Archon's Glory,common,{W},W,Instant,0,0,0,0,0,0,0,0,"Bargain (You may sacrifice an artifact, enchan...",$0.01
2,WOE,Armory Mice,common,{1}{W},W,Creature,Mouse,3,1,0,0,0,0,0,Celebration — Armory Mice gets +0/+2 as long a...,$0.01
3,WOE,Besotted Knight // Betroth the Beast,common,{3}{W},W,Creature,"Human, Knight",3,3,0,0,0,0,0,0,None
4,WOE,Break the Spell,common,{W},W,Instant,0,0,0,0,0,0,0,0,Destroy target enchantment. If a permanent you...,$0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,WOE,Become Brutes,uncommon,{1}{R},R,Sorcery,0,0,0,0,0,0,0,0,One or two target creatures each gain haste un...,$0.12
276,WOE,Charging Hooligan,uncommon,{3}{R},R,Creature,"Human, Peasant",3,3,0,0,0,0,0,"Whenever Charging Hooligan attacks, it gets +1...",$0.07
277,WOE,Ogre Chitterlord,rare,{4}{R}{R},R,Creature,"Ogre, Warrior",6,5,0,0,0,0,0,Menace\nWhenever Ogre Chitterlord enters the b...,$0.44
278,WOE,Intrepid Trufflesnout // Go Hog Wild,uncommon,{1}{G},G,Creature,Boar,3,1,0,0,0,0,0,"Whenever Intrepid Trufflesnout attacks alone, ...",None


In [20]:
#cur.execute(" ALTER TABLE RTR RENAME COLUMN 'Card 0me' TO Name;")
#con.commit()